In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

cp: cannot stat 'kaggle.json': No such file or directory


In [ ]:
!kaggle datasets download -d arnavsharmaas/chatbot-dataset-topical-chat

Dataset URL: https://www.kaggle.com/datasets/arnavsharmaas/chatbot-dataset-topical-chat
License(s): unknown
chatbot-dataset-topical-chat.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('/content/chatbot-dataset-topical-chat.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [ ]:
import pandas as pd

# Replace 'your_dataset.csv' with the path to your CSV file
df = pd.read_csv('/content/topical_chat.csv')

# Display the first few rows of the dataframe
df = df.head(4000)


In [ ]:
df.drop(columns=['sentiment', 'conversation_id'], inplace=True)
df.head()

,message
0,Are you a fan of Google or Microsoft?
1,Both are excellent technology they are helpfu...
2,"I'm not a huge fan of Google, but I use it a..."
3,Google provides online related services and p...
4,"Yeah, their services are good. I'm just not a..."


In [ ]:
df.shape

(4000, 1)

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer


In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['message'])

In [ ]:
tokenizer.word_index

{'i': 1,
 'the': 2,
 'a': 3,
 'that': 4,
 'to': 5,
 'of': 6,
 'you': 7,
 'is': 8,
 'it': 9,
 'in': 10,
 'and': 11,
 'was': 12,
 'have': 13,
 'they': 14,
 'know': 15,
 'for': 16,
 'do': 17,
 'like': 18,
 'are': 19,
 'he': 20,
 'did': 21,
 'not': 22,
 'but': 23,
 'be': 24,
 'with': 25,
 'think': 26,
 'so': 27,
 'on': 28,
 'about': 29,
 'would': 30,
 'as': 31,
 'there': 32,
 'yes': 33,
 'good': 34,
 'one': 35,
 'yeah': 36,
 'what': 37,
 'how': 38,
 'if': 39,
 'too': 40,
 "that's": 41,
 'really': 42,
 'can': 43,
 "it's": 44,
 'from': 45,
 'has': 46,
 'an': 47,
 'no': 48,
 'their': 49,
 'more': 50,
 'at': 51,
 'all': 52,
 "don't": 53,
 'well': 54,
 'some': 55,
 'when': 56,
 'lol': 57,
 'lot': 58,
 'his': 59,
 'my': 60,
 'them': 61,
 'me': 62,
 'interesting': 63,
 'were': 64,
 'been': 65,
 'or': 66,
 'though': 67,
 'just': 68,
 'by': 69,
 'movie': 70,
 'also': 71,
 'wonder': 72,
 'had': 73,
 'great': 74,
 'only': 75,
 'people': 76,
 'we': 77,
 "didn't": 78,
 'than': 79,
 'time': 80,
 'even':

In [ ]:
vocab_size = len(tokenizer.word_index)
vocab_size

6478

In [ ]:

X = []
for message in df['message']:
  for i in range(1, len(tokenizer.texts_to_sequences([message])[0])):
    X.append(tokenizer.texts_to_sequences([message])[0][:i+1])


In [ ]:
X[:10]

[[19, 7],
 [19, 7, 3],
 [19, 7, 3, 128],
 [19, 7, 3, 128, 6],
 [19, 7, 3, 128, 6, 182],
 [19, 7, 3, 128, 6, 182, 66],
 [19, 7, 3, 128, 6, 182, 66, 2547],
 [298, 19],
 [298, 19, 1410],
 [298, 19, 1410, 973]]

In [ ]:
longest_sen = max(len(i) for i in X)
longest_sen

87

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
X = pad_sequences(X, maxlen=longest_sen, padding='pre')

In [ ]:
y=X
X=X[:,:-1]
y=y[:,-1]

In [ ]:
X.shape

(72166, 86)

In [ ]:
y.shape

(72166,)

In [ ]:
from tensorflow.keras.utils import to_categorical
y = to_categorical(y, num_classes=vocab_size+1)

In [ ]:
y.shape

(72166, 6479)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense


In [ ]:
embedding_dim = 50
sequence_length = 86

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size+1, embedding_dim))
model.add(LSTM(128))
model.add(Dense(vocab_size+1, activation='softmax'))

# Build the model explicitly with the input shape
model.build(input_shape=(None, sequence_length))  # Here, None is the placeholder for any batch size


In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 86, 50)              │         323,950 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 128)                 │          91,648 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 6479)                │         835,791 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,251,389 (4.77 MB)

 Trainable params: 1,251,389 (4.77 MB)

 Non-trainable params: 0 (0.00 B)

In [26]:
model.fit(X, y, epochs=30, batch_size = 8)

Epoch 1/30
9021/9021 ━━━━━━━━━━━━━━━━━━━━ 87s 10ms/step - accuracy: 0.6682 - loss: 1.4266
Epoch 2/30
9021/9021 ━━━━━━━━━━━━━━━━━━━━ 140s 9ms/step - accuracy: 0.6739 - loss: 1.4151
Epoch 3/30
9021/9021 ━━━━━━━━━━━━━━━━━━━━ 141s 9ms/step - accuracy: 0.6757 - loss: 1.3948
Epoch 4/30


KeyboardInterrupt: 

In [27]:
model.save('/content/nexpre.keras')